# Example usage of GarNet

In [9]:
import numpy as np
import pandas as pd

# if you cloned the repo:
import sys
sys.path.append("../GarNet")

from importlib import reload
reload(garnet)

# Having issues importing despite module exising in virtual environment? 
# See https://github.com/jupyter/notebook/issues/397

<module 'garnet' from '../GarNet/garnet.py'>

# Map peaks to genes using a549 data

This is accomplished by intersecting motifs with genes specified in the open chromatin bed file. 

In [10]:
garnet_filepath = "../garnet_data/cisBP_hg19_SLIM/garnetDB_cisBP.hg19.LOD.10kb.chr1_SLIM.tsv"
peaks_filepath = "a549_example_peaks.chr1.bed"

df = garnet.map_peaks(peaks_filepath, garnet_filepath)
df.head()

,chrom,start,end,motifName,motifLODScore,motifStrand,geneName,geneStart,geneEnd,motif_gene_distance
0,chr1,565301,565308,HOXA4,7.539123,-,JA429830,566114,566115,813
1,chr1,565460,565467,HOXA4,7.539123,-,JA429830,566114,566115,654
2,chr1,568230,568241,GATA1,10.237757,-,JA429830,566114,566115,-2116
3,chr1,568231,568240,LMO2,8.409497,-,JA429830,566114,566115,-2117
4,chr1,570229,570235,HOXA4,7.118708,+,JA429830,566114,566115,-4121


# Regress gene expression against TF Motif binding score

Perform regression, plots are automatically generated in `regression_plots/`. 

In [11]:
expression_filepath = "a549_example_expression.txt"

results = garnet.TF_regression(df, expression_filepath, ".")
results.head(10)

02:53:45 - GarNet: INFO - Performing linear regression for 7 transcription factor expression profiles...
02:53:45 - GarNet: INFO - Performing linear regression for 7 transcription factor expression profiles...
02:53:45 - GarNet: INFO - Performing linear regression for 7 transcription factor expression profiles...
02:53:45 - GarNet: INFO - Performing linear regression for 7 transcription factor expression profiles...


,Transcription Factor,Slope,P-Value,Targets
0,ATF5,0.247165,0.234486,"ACOT7,CSF1,SOX13,PBXIP1,PTGS2,ID3,IL6R,KIRREL,..."
4,HOXA4,0.149229,0.443636,"IER5,LMNA,TGFB2,TMEM63A,RAB25,AMPD2,ATF3,EPB41..."
3,GATA1,0.149975,0.491204,"ELF3,S100A4,PLEKHO1,KIAA1522,CYR61,NIPAL3,ARHG..."
6,SREBF1,0.134617,0.665189,"IER5,THBS3,KIAA1522,PIK3C2B,SAMD11,EPB41,GSTM3"
2,EGR3,-0.064156,0.727905,"IER5,AMPD2,PLEKHO1,THBS3,ACOT7,EFNA1,ATF3,CYR6..."
1,CEBPB,0.070856,0.834070,"DHCR24,PIK3C2B,NBL1,NIPAL3,ST6GALNAC3,CYR61,KCNK1"
5,LMO2,-0.006344,0.959221,"ZBTB7B,S100A4,PVRL4,CAMK2N1,PLEKHO1,F11R,RASSF..."


# Appendix

## Download and format reference file

### 1. Download UCSC known genes file

Visit [UCSC Table Browser](https://genome.ucsc.edu/cgi-bin/hgTables?command=start) and select the following options

 - **assembly**: Feb. 2009 (GRCh37/hg19)
 - **group**: Genes and Gene Predictions
 - **track**: UCSC Genes
 - **table**: knownGene
 - **region**: genome
 - **output format**: selected fields from primary and related tables

Press "get output", clear all fields, then check the following fields:

 - chrom
 - strand
 - txStart
 - txEnd
 - geneSymbol
 
Press "get output" again, and save the table locally. A copy (retrieved 9/11/2017) is saved in `../garnet_data/reference/ucsc_known_genes.hg19.tsv`.

### 2. Reformat to BED file

In [ ]:
ucsc_reference = pd.read_csv("../garnet_data/reference/ucsc_known_genes.hg19.tsv", sep='\t', names=["chrom", "strand", "chromStart", "chromEnd", "name"], comment="#")
ucsc_reference.drop_duplicates(inplace=True)
ucsc_reference["score"] = "."
ucsc_reference.head()

In [ ]:
ucsc_reference_bed = ucsc_reference[["chrom", "chromStart", "chromEnd", "name", "score", "strand"]]
ucsc_reference_bed.head()

In [ ]:
ucsc_reference_bed.to_csv("../garnet_data/reference/ucsc_reference.hg19.bed", sep='\t', header=False, index=False)